# 🚨 URGENT: 40-MINUTE SUBMISSION DEADLINE 🚨

## ⚡ QUICK START GUIDE

**IMMEDIATE ACTIONS:**
1. **🎯 Set GPU NOW**: Runtime → Change runtime type → T4 GPU → Save
2. **▶️ Run ALL cells** in order (takes ~25 minutes total)
3. **📥 Download model** when training completes
4. **🚀 Deploy to Streamlit Cloud** while training runs

**OPTIMIZED SETTINGS:**
- ✅ Reduced to **15 epochs** (was 50)
- ✅ Increased **batch size** to 256
- ✅ Higher **learning rate** for faster convergence
- ✅ **Total time: ~25 minutes** (fits in 40-minute deadline)

**PARALLEL TASKS:**
- Start training NOW
- While training runs, set up GitHub repo
- Upload your other files to GitHub
- Prepare Streamlit Cloud deployment

---

# MNIST Handwritten Digit Generation Training
## Conditional GAN Training on Google Colab with T4 GPU

**Instructions:**
1. Make sure Runtime → Change runtime type → T4 GPU is selected
2. Run all cells in order
3. Training will take about 30-45 minutes
4. Download the final `generator_model.pth` file

In [ ]:
# Check if GPU is available
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU not available. Please change runtime type to T4 GPU")

In [ ]:
# Install required packages (if needed)
!pip install torch torchvision matplotlib numpy pillow

In [ ]:
# Create model.py file
%%writefile model.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self, noise_dim=100, num_classes=10):
        super(Generator, self).__init__()
        self.noise_dim = noise_dim
        self.num_classes = num_classes
        
        # Label embedding
        self.label_embedding = nn.Embedding(num_classes, 50)
        
        # Generator network
        self.fc1 = nn.Linear(noise_dim + 50, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, 28 * 28)
        
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, noise, labels):
        # Embed labels
        label_embed = self.label_embedding(labels)
        
        # Concatenate noise and label embedding
        x = torch.cat([noise, label_embed], dim=1)
        
        # Forward pass
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = self.dropout(x)
        x = torch.tanh(self.fc4(x))
        
        # Reshape to image format
        x = x.view(-1, 1, 28, 28)
        return x

class Discriminator(nn.Module):
    def __init__(self, num_classes=10):
        super(Discriminator, self).__init__()
        self.num_classes = num_classes
        
        # Label embedding
        self.label_embedding = nn.Embedding(num_classes, 50)
        
        # Discriminator network
        self.fc1 = nn.Linear(28 * 28 + 50, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)
        
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, images, labels):
        # Flatten images
        images = images.view(-1, 28 * 28)
        
        # Embed labels
        label_embed = self.label_embedding(labels)
        
        # Concatenate image and label embedding
        x = torch.cat([images, label_embed], dim=1)
        
        # Forward pass
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = self.dropout(x)
        x = torch.sigmoid(self.fc4(x))
        
        return x

def create_models():
    """Create and return generator and discriminator models"""
    generator = Generator()
    discriminator = Discriminator()
    return generator, discriminator

In [ ]:
# Import required libraries and setup
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from model import Generator, Discriminator

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Hyperparameters - OPTIMIZED FOR FAST TRAINING (40 MINUTES)
BATCH_SIZE = 256  # Increased for faster training
LEARNING_RATE = 0.0003  # Slightly higher for faster convergence
NUM_EPOCHS = 15  # Reduced from 50 to 15 for speed
NOISE_DIM = 100
NUM_CLASSES = 10

print(f"⚡ FAST TRAINING Configuration:")
print(f"- Batch size: {BATCH_SIZE} (larger for speed)")
print(f"- Learning rate: {LEARNING_RATE} (higher for faster convergence)")
print(f"- Epochs: {NUM_EPOCHS} (reduced for time constraint)")
print(f"- Estimated training time: {NUM_EPOCHS * 1.5:.0f} minutes")

In [ ]:
# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

train_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform
)

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

print(f"Dataset loaded: {len(train_dataset)} training samples")
print(f"Number of batches: {len(train_loader)}")

In [ ]:
# Initialize models
generator = Generator(NOISE_DIM, NUM_CLASSES).to(device)
discriminator = Discriminator(NUM_CLASSES).to(device)

# Loss function and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

print(f"Generator parameters: {sum(p.numel() for p in generator.parameters()):,}")
print(f"Discriminator parameters: {sum(p.numel() for p in discriminator.parameters()):,}")
print("Models initialized and moved to GPU!")

In [ ]:
# Training function with progress tracking - OPTIMIZED FOR SPEED
def train_gan():
    generator.train()
    discriminator.train()
    
    # Lists to track losses
    g_losses = []
    d_losses = []
    
    for epoch in range(NUM_EPOCHS):
        epoch_g_loss = 0
        epoch_d_loss = 0
        
        for i, (real_images, real_labels) in enumerate(train_loader):
            batch_size = real_images.size(0)
            real_images = real_images.to(device)
            real_labels = real_labels.to(device)
            
            # Labels for real and fake data
            real_target = torch.ones(batch_size, 1).to(device)
            fake_target = torch.zeros(batch_size, 1).to(device)
            
            # Train Discriminator
            optimizer_D.zero_grad()
            
            # Real data
            real_output = discriminator(real_images, real_labels)
            real_loss = criterion(real_output, real_target)
            
            # Fake data
            noise = torch.randn(batch_size, NOISE_DIM).to(device)
            fake_labels = torch.randint(0, NUM_CLASSES, (batch_size,)).to(device)
            fake_images = generator(noise, fake_labels)
            fake_output = discriminator(fake_images.detach(), fake_labels)
            fake_loss = criterion(fake_output, fake_target)
            
            # Total discriminator loss
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_D.step()
            
            # Train Generator
            optimizer_G.zero_grad()
            
            # Generate fake images and get discriminator output
            fake_output = discriminator(fake_images, fake_labels)
            g_loss = criterion(fake_output, real_target)
            
            g_loss.backward()
            optimizer_G.step()
            
            # Accumulate losses
            epoch_g_loss += g_loss.item()
            epoch_d_loss += d_loss.item()
            
            # Print progress less frequently for speed
            if i % 50 == 0:  # Reduced from 100 to 50
                print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i+1}/{len(train_loader)}], '
                      f'D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')
        
        # Track average losses
        avg_g_loss = epoch_g_loss / len(train_loader)
        avg_d_loss = epoch_d_loss / len(train_loader)
        g_losses.append(avg_g_loss)
        d_losses.append(avg_d_loss)
        
        print(f'✅ Epoch [{epoch+1}/{NUM_EPOCHS}] completed - Avg D_loss: {avg_d_loss:.4f}, Avg G_loss: {avg_g_loss:.4f}')
        
        # Save sample images every 5 epochs (reduced from 10)
        if (epoch + 1) % 5 == 0:
            save_sample_images(epoch + 1)
    
    return g_losses, d_losses

def save_sample_images(epoch):
    """Save sample generated images"""
    generator.eval()
    with torch.no_grad():
        # Generate one image for each digit
        noise = torch.randn(10, NOISE_DIM).to(device)
        labels = torch.arange(0, 10).to(device)
        fake_images = generator(noise, labels)
        
        # Denormalize images
        fake_images = fake_images * 0.5 + 0.5
        
        # Create subplot
        fig, axes = plt.subplots(2, 5, figsize=(12, 6))
        for i in range(10):
            row = i // 5
            col = i % 5
            axes[row, col].imshow(fake_images[i].cpu().squeeze(), cmap='gray')
            axes[row, col].set_title(f'Digit {i}')
            axes[row, col].axis('off')
        
        plt.suptitle(f'Generated Images - Epoch {epoch}', fontsize=16)
        plt.tight_layout()
        plt.show()
    
    generator.train()

print("⚡ FAST Training functions defined. Ready to start training!")

In [ ]:
# Start training - FAST MODE FOR SUBMISSION DEADLINE
print("🚀 Starting FAST GAN training...")
print("⏰ OPTIMIZED FOR 40-MINUTE DEADLINE")
print("This will take approximately 20-25 minutes on T4 GPU")
print("Sample images will be shown every 5 epochs")
print("=" * 60)

import time
start_time = time.time()

g_losses, d_losses = train_gan()

end_time = time.time()
training_time = (end_time - start_time) / 60

print(f"\n🎉 Training completed in {training_time:.1f} minutes!")
print("🚀 Ready for deployment!")

In [ ]:
# Save the trained model
torch.save(generator.state_dict(), 'generator_model.pth')
print("✅ Model saved as 'generator_model.pth'")

# Plot training losses
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(g_losses, label='Generator Loss')
plt.plot(d_losses, label='Discriminator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Losses')
plt.legend()
plt.grid(True)

# Generate final test images
plt.subplot(1, 2, 2)
generator.eval()
with torch.no_grad():
    # Generate 5 images of digit 7 as final test
    noise = torch.randn(5, NOISE_DIM).to(device)
    labels = torch.full((5,), 7).to(device)  # Generate digit 7
    test_images = generator(noise, labels)
    test_images = test_images * 0.5 + 0.5
    
    # Show the 5 generated images
    for i in range(5):
        plt.subplot(2, 5, i + 6)
        plt.imshow(test_images[i].cpu().squeeze(), cmap='gray')
        plt.title(f'Generated 7 #{i+1}')
        plt.axis('off')

plt.tight_layout()
plt.show()

print("\n📊 Training summary:")
print(f"Final Generator Loss: {g_losses[-1]:.4f}")
print(f"Final Discriminator Loss: {d_losses[-1]:.4f}")
print(f"Total training epochs: {NUM_EPOCHS}")

In [ ]:
# Final test: Generate images for all digits
print("🎯 Final Test: Generating images for all digits (0-9)")

generator.eval()
with torch.no_grad():
    # Generate 2 images for each digit
    fig, axes = plt.subplots(2, 10, figsize=(20, 4))
    
    for digit in range(10):
        noise = torch.randn(2, NOISE_DIM).to(device)
        labels = torch.full((2,), digit).to(device)
        digit_images = generator(noise, labels)
        digit_images = digit_images * 0.5 + 0.5
        
        for i in range(2):
            axes[i, digit].imshow(digit_images[i].cpu().squeeze(), cmap='gray')
            axes[i, digit].set_title(f'Digit {digit}')
            axes[i, digit].axis('off')
    
    plt.suptitle('Final Generated Images - All Digits', fontsize=16)
    plt.tight_layout()
    plt.show()

print("\n✅ Model training and testing completed!")
print("\n📥 Next steps:")
print("1. Download 'generator_model.pth' from the files panel (left sidebar)")
print("2. Upload it to your Streamlit app repository")
print("3. Deploy your app on Streamlit Cloud")

In [ ]:
# Download instructions
from google.colab import files

print("📥 Downloading the trained model...")
files.download('generator_model.pth')
print("✅ Download started! Check your browser's download folder.")